In [1]:
import glob
import h5py
import numpy as np
from fastai.basics import *
from fastai.callback.schedule import fit_one_cycle
from fastai.callback.hook import summary
from fastai.callback.tracker import SaveModelCallback
from fastai.callback.fp16 import *
from fastai.data.load import DataLoader as FastDataLoader
from fastai.callback.progress import CSVLogger
from torch.utils.data import *
from collections.abc import Iterable

import utils.config as config
from utils.w4c_dataloader_aio import create_dataset

# Parameters

In [2]:
region_ids = ['R1', 'R2', 'R3']
bs = 12
num_workers = 3
device = 'cuda'
folder_to_save_models = 'weights'

# 1. Initialise Dataset and Dataloaders

In [3]:
def get_datasets(region_ids, train_on_validation=False):
    if len(region_ids)==1:
        params = config.get_params(region_id=region_ids[0])['data_params']
        params['return_meta']=False
        params['static_path']=
        valid = create_dataset('training', params)
        if train_on_validation:
            train = [create_dataset('training', params), create_dataset('validation', params)]
            train = ConcatDataset(train)
            train.n_inp = 1
        else:
            train = create_dataset('training', params)
    else:
        train=[]
        valid=[]
        for reg in region_ids:
            params = config.get_params(region_id=reg)['data_params']
            train.append(create_dataset('training', params))
            if train_on_validation:
                train.append(create_dataset('validation', params))
            valid.append(create_dataset('validation', params))
        train = ConcatDataset(train)
        train.n_inp = 1
        valid = ConcatDataset(valid)
        valid.n_inp = 1
    return train, valid

In [4]:
train, valid = get_datasets(region_ids, train_on_validation=False)

Using data for region R1 | size: 256 | Nile Region


In [5]:
train_dl = FastDataLoader(dataset=train,
                          bs=bs,
                          num_workers=num_workers,
                          pin_memory=True,
                          shuffle=True,
                          device=torch.device(device))
valid_dl = FastDataLoader(dataset=valid,
                          bs=bs,
                          num_workers=num_workers,
                          pin_memory=True,
                          shuffle=True,
                          device=torch.device(device))
data = DataLoaders(train_dl, valid_dl, device=torch.device(device))

# 2. Load Baseline U-Net

In [9]:
from utils.vunet_model import Net1, VUNetLoss2, valid_leaderboard, valid_leaderboard2
Model = Net1(in_channels=8*4+3,out_channels=4*32)

In [10]:
learn = Learner(data, Model.to(device), loss_func=VUNetLoss2, metrics=[valid_leaderboard, valid_leaderboard2], model_dir=folder_to_save_models, cbs=CSVLogger)

In [11]:
learn.summary()

/Users/qiqi/miniconda3/envs/w4c/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459044803/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Net1 (Input shape: 12)
Layer (type)         Output Shape         Param #    Trainable 
                     12 x 64 x 256 x 256 
Conv2d                                    20224      True      
ELU                                                            
GroupNorm                                 128        True      
Dropout2d                                                      
____________________________________________________________________________
                     12 x 64 x 256 x 256 
Conv2d                                    57088      True      
ELU                                                            
GroupNorm                                 128        True      
Dropout2d                                                      
____________________________________________________________________________
                     12 x 64 x 256 x 256 
Conv2d                                    93952      True      
ELU                                                      

In [9]:
#lr_min,lr_steep = learn.lr_find()
#print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [13]:
learn.fit_one_cycle(2, lr_max=2e-04)

In [10]:
learn.save('Comb_2')

Path('/storage/weather4cast/models/Comb_2.pth')

In [9]:
#learn.load('Comb_2')

In [10]:
learn.fit_flat_cos(2, lr=2e-04, pct_start=0)

epoch,train_loss,valid_loss,valid_leaderboard,valid_leaderboard2,time
0,0.441131,0.426516,0.701760,0.426097,2:27:41
1,0.426014,0.413974,0.693612,0.413958,2:52:13


In [11]:
learn.save('Comb_4')

Path('/storage/weather4cast/models/Comb_4.pth')

In [9]:
#learn.load('Comb_4')

In [11]:
learn.fit_flat_cos(2, lr=2e-04, pct_start=0)

epoch,train_loss,valid_loss,valid_leaderboard,valid_leaderboard2,time
0,0.397676,0.414760,0.698782,0.414678,2:02:00
1,0.393578,0.403136,0.684215,0.403118,2:26:25


In [12]:
learn.save('Comb_6')

Path('/storage/weather4cast/models/Comb_6.pth')

In [9]:
#learn.load('Comb_6')

In [10]:
learn.fit_flat_cos(2, lr=2e-04, pct_start=0)

epoch,train_loss,valid_loss,valid_leaderboard,valid_leaderboard2,time
0,0.392610,0.409428,0.687157,0.409365,2:04:13
1,0.384217,0.398677,0.680336,0.398660,2:05:42


In [11]:
learn.save('Comb_8')

Path('/storage/weather4cast/models/Comb_8.pth')

In [9]:
#learn.load('Comb_8')

In [10]:
learn.fit_flat_cos(2, lr=2e-04, pct_start=0)

epoch,train_loss,valid_loss,valid_leaderboard,valid_leaderboard2,time
0,0.367376,0.400800,0.680977,0.400765,2:53:45
1,0.372932,0.395273,0.678566,0.395255,3:06:35


In [11]:
learn.save('Comb_10')

Path('/storage/weather4cast/models/Comb_10.pth')

In [ ]:
#learn.load('Comb_10')

In [13]:
learn.fit_flat_cos(2, lr=2e-04, pct_start=0)

epoch,train_loss,valid_loss,valid_leaderboard,valid_leaderboard2,time
0,0.364000,0.400641,0.682575,0.400606,3:10:17
1,0.370470,0.391266,0.676537,0.391249,3:10:44


In [14]:
learn.save('Comb_12')

Path('/storage/weather4cast/models/Comb_12.pth')

# 3. Train with Validation Data

In [ ]:
train, valid = get_datasets(region_ids, train_on_validation=True)

In [ ]:
train_dl = FastDataLoader(dataset=train,
                          bs=bs,
                          num_workers=num_workers,
                          pin_memory=True,
                          shuffle=True,
                          device=torch.device(device))
valid_dl = FastDataLoader(dataset=valid,
                          bs=bs,
                          num_workers=num_workers,
                          pin_memory=True,
                          shuffle=True,
                          device=torch.device(device))
data = DataLoaders(train_dl, valid_dl, device=torch.device(device))

In [ ]:
learn = Learner(
    data, 
    Model.to(device), 
    loss_func=VUNetLoss2, metrics=[valid_leaderboard, valid_leaderboard2], 
    model_dir=folder_to_save_models, cbs=CSVLogger(append=True))

In [ ]:
learn.summary()

In [11]:
#learn.load('Comb_12')

In [12]:
learn.fit_flat_cos(2, lr=2e-04, pct_start=0)

epoch,train_loss,valid_loss,valid_leaderboard,valid_leaderboard2,time
0,0.369952,0.378295,0.667222,0.378240,2:03:36
1,0.358378,0.367179,0.653086,0.367161,2:03:15


In [13]:
learn.save('Lomb_14')

Path('/storage/weather4cast/models/Lomb_14.pth')